In [21]:
import os

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from pytorch_tabular.utils import make_mixed_dataset, print_metrics

# %load_ext autoreload
# %autoreload 2

ImportError: cannot import name 'print_metrics' from 'pytorch_tabular.utils' (/home/manujosephv/pytorch_tabular/src/pytorch_tabular/utils/__init__.py)

First of all, let's create a synthetic data which is a mix of numerical and categorical features

In [ ]:
data, cat_col_names, num_col_names = make_mixed_dataset(
    task="classification", n_samples=10000, n_features=20, n_categories=4
)
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

# Importing the Library

In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig

## Define the Configs

This is the most crucial step in the process. There are four configs that you need to provide(most of them have intelligent default values), which will drive the rest of the process.

* DataConfig - Define the target column names, categorical and numerical column names, any transformation you need to do, etc.
* ModelConfig - There is a specific config for each of the models. This determines which model we are going to train and also lets you define the hyperparameters of the model
* TrainerConfig - This let's you configure the training process by setting things like batch_size, epochs, early stopping, etc. The vast majority of parameters are directly borrowed from PyTorch Lightning and is passed to the underlying Trainer object during training
* OptimizerConfig - This let's you define and use different Optimizers and LearningRate Schedulers. Standard PyTorch Optimizers and Learning RateSchedulers are supported. For custom optimizers, you can use the parameter in the fit method to overwrite this. The custom optimizer should be PyTorch compatible
* ExperimentConfig - This is an optional parameter. If set, this defines the Experiment Tracking. Right now, only two experiment tracking frameworks are supported: Tensorboard and Weights&Biases. W&B experiment tracker has more features like tracking the gradients and logits across epochs.

In [ ]:
data_config = DataConfig(
    target=[
        "target"
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    accelerator="auto",  # can be 'cpu','gpu', 'tpu', or 'ipu'
)
optimizer_config = OptimizerConfig()

# Best practice: Use head config to define the head. This will help you not pass
# unnecessary params to the model
head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="32-16",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    dropout=0.1,
    initialization="kaiming",
    head="LinearHead",  # Linear Head
    head_config=head_config,  # Linear Head Config
    learning_rate=1e-3,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-12-26 13:08:43,641 - {pytorch_tabular.tabular_model:134} - INFO - Experiment Tracking is turned off

## Training the Model 
Now that we have defined the configs and the TabularModel. We just need to call the `fit` method and pass the train and test dataframes. We can also pass in validation dataframe. But if omitted, TabularModel will separate 20%(also configurable) at random from the data as validation.

By default, EarlyStopping is enabled and is monitoring Validation Loss with a patience of 3 epochs. The trainer also saves the best model(based on validation loss) and loads that model at the end of training. `TrainerConfig` has the parameters to tweak this default behaviour.

In [22]:
tabular_model.fit(train=train, validation=val)

Seed set to 42


2023-12-27 15:08:36,554 - {pytorch_tabular.tabular_model:506} - INFO - Preparing the DataLoaders

2023-12-27 15:08:36,797 - {pytorch_tabular.tabular_datamodule:431} - INFO - Setting up the datamodule for          
classification task

2023-12-27 15:08:37,050 - {pytorch_tabular.tabular_model:556} - INFO - Preparing the Model: CategoryEmbeddingModel

2023-12-27 15:08:37,208 - {pytorch_tabular.tabular_model:322} - INFO - Preparing the Trainer

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  1.5 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │     92 │
│ 2 │ head             │ LinearHead                │     34 │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 1.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-12-27 15:08:47,929 - {pytorch_tabular.tabular_model:645} - INFO - Training the model completed

2023-12-27 15:08:47,930 - {pytorch_tabular.tabular_model:1491} - INFO - Loading the best model

## Evaluating the Model

### Loss and Metrics on New Data
To evaluate the model on new data on the same metrics/loss that was used during training, we can use the `evaluate` method

In [23]:
result = tabular_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9368000030517578     │
│         test_loss         │    0.1678159385919571     │
└───────────────────────────┴───────────────────────────┘

/home/manujosephv/miniconda3/envs/lightning_upgrade/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


## New Predictions as DataFrame
To get the prediction as a dataframe, we can use the `predict` method. This will add predictions to the same dataframe that was passed in. For classification problems, we get both the probabilities and the final prediction taking 0.5 as the threshold

In [24]:
pred_df = tabular_model.predict(test)
pred_df.head()

,class_0_probability,class_1_probability,prediction
6252,0.006235,0.993765,class_1
4684,0.438156,0.561844,class_1
1731,0.998281,0.001719,class_0
4742,0.605581,0.394419,class_0
4521,0.019349,0.980651,class_1


In [25]:
print_metrics(
    metrics=[
        (accuracy_score, "Accuracy", {}),
        (f1_score, "F1", {"average": "macro"}),
    ],
    y_true=test["target"],
    y_pred=pred_df["prediction"],
    tag="Holdout",
)

Holdout Accuracy: 0.9368 | Holdout F1: 0.9367998382075857


## Saving and Loading the Model

In [26]:
tabular_model.save_model("examples/basic")

2023-12-27 15:08:59,030 - {pytorch_tabular.tabular_model:1535} - WARNING - Directory is not empty. Overwriting the 
contents.

In [27]:
loaded_model = TabularModel.load_model("examples/basic")

2023-12-27 15:08:59,273 - {pytorch_tabular.tabular_model:159} - INFO - Experiment Tracking is turned off

2023-12-27 15:08:59,277 - {pytorch_tabular.tabular_model:322} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
result = loaded_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/home/manujosephv/miniconda3/envs/lightning_upgrade/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9368000030517578     │
│         test_loss         │    0.1678159385919571     │
└───────────────────────────┴───────────────────────────┘